In [1]:
# Cell 1: imports & config
import os, time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import seaborn as sns

# Config
DATA_ROOT = "../fer2013"    # change if your extracted folder is elsewhere
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
TEST_DIR  = os.path.join(DATA_ROOT, "test")

NUM_EPOCHS = 8              # start small; increase if you have time
BATCH_SIZE = 64
LR = 1e-4
NUM_WORKERS = 4             # set 0 on Windows if you get issues
SEED = 42

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)
np.random.seed(SEED)

print("Device:", device)


Device: cuda


In [2]:
# Cell 2: Data transforms and loaders
transform_train = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=transform_train)
test_dataset = datasets.ImageFolder(TEST_DIR, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

print(f"Train samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Classes: {train_dataset.classes}")


Train samples: 28709
Test samples: 7178
Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [6]:
# Cell 3: Load pretrained MobileNetV2 and modify for 7 classes

mobilenet = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)

# Modify the final classification layer for 7 emotion classes
num_ftrs = mobilenet.classifier[1].in_features
mobilenet.classifier[1] = nn.Linear(num_ftrs, 7)

mobilenet = mobilenet.to(device)

# # Phase 1: Freeze all feature extractor layers (only train classifier)
# for param in mobilenet.features.parameters():
#     param.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet.parameters(), lr=1e-4, weight_decay=1e-5)


print("✅ Model ready for Phase 1 training (classifier only)")


✅ Model ready for Phase 1 training (classifier only)


In [7]:
# Cell 4: Training setup and loop
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mobilenet.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

EPOCHS = 10  # Increase if GPU allows
best_acc = 0.0

for epoch in range(EPOCHS):
    mobilenet.train()
    train_loss = 0.0
    correct, total = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = mobilenet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total

    # Validation step
    mobilenet.eval()
    val_loss = 0.0
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = mobilenet(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = 100 * correct / total

    print(f"Epoch [{epoch+1}/{EPOCHS}] | "
          f"Train Loss: {train_loss/len(train_loader):.4f} | "
          f"Val Loss: {val_loss/len(test_loader):.4f} | "
          f"Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(mobilenet.state_dict(), "best_mobilenetv2.pth")

    scheduler.step()

print(f"\n✅ Training complete! Best Val Accuracy: {best_acc:.2f}%")


Epoch [1/10] | Train Loss: 1.2109 | Val Loss: 1.0301 | Train Acc: 54.05% | Val Acc: 60.74%
Epoch [2/10] | Train Loss: 0.9502 | Val Loss: 0.9701 | Train Acc: 64.27% | Val Acc: 63.33%
Epoch [3/10] | Train Loss: 0.8314 | Val Loss: 0.9396 | Train Acc: 69.00% | Val Acc: 65.06%
Epoch [4/10] | Train Loss: 0.7341 | Val Loss: 0.9265 | Train Acc: 73.15% | Val Acc: 66.45%
Epoch [5/10] | Train Loss: 0.6284 | Val Loss: 0.9904 | Train Acc: 77.02% | Val Acc: 65.77%
Epoch [6/10] | Train Loss: 0.4545 | Val Loss: 0.9673 | Train Acc: 84.55% | Val Acc: 67.41%
Epoch [7/10] | Train Loss: 0.3959 | Val Loss: 0.9953 | Train Acc: 86.75% | Val Acc: 67.15%
Epoch [8/10] | Train Loss: 0.3639 | Val Loss: 1.0189 | Train Acc: 88.19% | Val Acc: 67.00%
Epoch [9/10] | Train Loss: 0.3302 | Val Loss: 1.0536 | Train Acc: 89.16% | Val Acc: 67.47%
Epoch [10/10] | Train Loss: 0.3066 | Val Loss: 1.0795 | Train Acc: 90.04% | Val Acc: 67.08%

✅ Training complete! Best Val Accuracy: 67.47%


In [9]:
# ✅ Cell 6: Save the fine-tuned MobileNetV2 model
save_path = "../trained_models/mobilenetv2_finetuned.pth"

torch.save(mobilenet.state_dict(), save_path)

print(f"✅ Fine-tuned MobileNetV2 saved successfully at: {save_path}")


✅ Fine-tuned MobileNetV2 saved successfully at: ../trained_models/mobilenetv2_finetuned.pth
